In [27]:
import docker
from IPython.display import display_html # Jupyter
import logging
import sys
import inspect


# Configure logging
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logger = logging.getLogger(__name__)

def create_dockerfile(requirements_file="requirements.txt", script_name="app.py", workdir="/app", base_image="python:3.8-slim", function_args=""):
    """
    Create a Dockerfile with the specified configuration.

    Args:
        requirements_file (str): Path to the requirements file.
        script_name (str): Name of the script to run in the Docker container.
        workdir (str): Working directory inside the container.
        base_image (str): Base Docker image.

    Returns:
        None
    """
    dockerfile_content = f"""
        FROM {base_image}
        WORKDIR {workdir}
        COPY . {workdir}
        RUN pip install --no-cache-dir -r {requirements_file}
        EXPOSE 80
        CMD ["python3", "-c", "import {script_name}; {script_name}.{script_name}({function_args})"]
    """
    
    with open("Dockerfile", "w") as dockerfile:
        dockerfile.write(dockerfile_content)

def build_docker_image(image_name):
    """
    Build a Docker image with the specified name.

    Args:
        image_name (str): Name of the Docker image.

    Returns:
        tuple: A tuple containing the Docker image and build logs.
    """
    from IPython.display import display_html
    client = docker.from_env()

    logger.info(f"Building Docker image: {image_name}")
    return client.images.build(path=".", tag=image_name)

def run_docker_container_interactive(image, ports, detach=True):
    """
    Run a Docker container interactively based on the given image.

    Args:
        image (str): ID or name of the Docker image.
        ports (dict): Port mapping for the container.
        detach (bool): Whether to run the container in the background.

    Returns:
        docker.models.containers.Container: The running Docker container.
    """
    client = docker.from_env()

    logger.info(f"Running Docker image interactively: {image}")
    container = client.containers.run(image=image, ports=ports, detach=detach, tty=True, stdin_open=True)

    return container

def stream_container_logs(container):
    """
    Stream logs from a Docker container.

    Args:
        container (docker.models.containers.Container): The Docker container.

    Returns:
        None
    """
    logger.info("Attaching to container logs:")
    current_line = ""

    for log_chunk in container.logs(stream=True, follow=True):
        log_chunk_decoded = log_chunk.decode("utf-8")
        
        for char in log_chunk_decoded:
            if char == '\n':
                # Print the accumulated line when a newline is encountered
                print(current_line.strip())
                current_line = ""
            else:
                # Accumulate characters until a newline is encountered
                current_line += char


def create_requirements_txt():
    """
    Create a requirements.txt file if it doesn't exist.

    Returns:
        None
    """
    try:
        with open("requirements.txt", "r"):
            pass  # If the file exists, do nothing
    except FileNotFoundError:
        with open("requirements.txt", "w") as requirements_file:
            requirements_file.write("numpy==1.18.5\n")
            requirements_file.write("requests==2.24.0\n")
            # Add other dependencies as needed

def get_function_source(func_name):
    """
    Get the source code of a function.

    Args:
        func_name (str): Name of the function.

    Returns:
        str: Source code of the function.
    """
    try:
        func = globals()[func_name]
        return inspect.getsource(func)
    except KeyError:
        return f"Function '{func_name}' not found"

def function_to_py(func_name):
    """
    Convert a function to a Python file.

    Args:
        func_name (str): Name of the function.

    Returns:
        None
    """
    content = get_function_source(func_name)
    file_path = f"{func_name}.py"
    try:
        with open(file_path, 'w') as file:
            file.write(content)
        print(f"Content successfully written to {file_path}")
    except Exception as e:
        print(f"Error writing to {file_path}: {str(e)}")


def dockerize_function(func_name):
    docker_image_name = f"{func_name}_image"
    
    # Generate the Python script
    function_to_py(func_name)

    # Create other necessary filesCMD
    create_requirements_txt()
    create_dockerfile(script_name=func_name)

    # Build the Docker image
    image, _ = build_docker_image(docker_image_name)

    # Run the Docker container
    container = run_docker_container_interactive(image.id, ports={'80/tcp': 4000}, detach=True)
    logger.info(f"Docker container ID: {container.id}")

    # Stream container logs
    stream_container_logs(container)
    
    # Read the output from the container
    result = container.logs().decode("utf-8").strip()
    
    # Display the result within Jupyter Notebook
    display_html(f"<p>{result}</p>", raw=True)
    
    return result

def foo():
    print("foo")


if __name__ == "__main__":
    # Configure logging for the script
    logging.basicConfig(stream=sys.stdout, level=logging.INFO)
    logger = logging.getLogger(__name__)
    
    # Execute the dockerize_function and display the output within Jupyter Notebook
    dockerize_function("foo")

Content successfully written to foo.py
INFO:__main__:Building Docker image: foo_image


INFO:__main__:Running Docker image interactively: sha256:ab012d01d908c3b068e17b75b30e76da8a9b5c96da308b56f06e9a458fa0e24a
INFO:__main__:Docker container ID: fc0257fbc22e451962a2b27679aba0baedec39067d52ab1963378eca6a789535
INFO:__main__:Attaching to container logs:
foo


foo